In [1]:
import os
import textwrap
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [7]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
from datasets import load_dataset
ds = load_dataset(
    "hwaseem04/Aya-testing",
    data_files={"xGQA_vqa": "data/xGQA_vqa-00000-of-00001.parquet"}
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


xGQA_vqa-00000-of-00001.parquet:   0%|          | 0.00/340M [00:00<?, ?B/s]

Generating xGQA_vqa split: 0 examples [00:00, ? examples/s]

In [8]:
ds['xGQA_vqa'][0]


{'sample_id': 1,
 'image_id': 'n161313',
 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x280>,
 'question_en': 'Is it overcast?',
 'answer_en': 'No, it is clear.',
 'question_bn': 'এটা কি মেঘাচ্ছন্ন?',
 'answer_bn': 'না, এটা স্পষ্ট।',
 'question_de': 'Ist es bewölkt?',
 'answer_de': 'Nein, es ist klar.',
 'question_id': 'Apakah langit mendung ?',
 'answer_id': 'Tidak, itu jelas.',
 'question_ko': '날이 흐린가요?',
 'answer_ko': '아니, 분명해요.',
 'question_ru': 'Это пасмурная погода?',
 'answer_ru': 'Нет, это ясно.',
 'question_zh': '是阴天吗？',
 'answer_zh': '不，很清楚。'}

In [11]:
from tqdm import tqdm
from PIL import Image
import torch
from transformers import PaliGemmaProcessor, PaliGemmaForConditionalGeneration
from transformers.image_utils import load_image

# Load the PaliGemma 2 model and processor
model_id = "google/paligemma2-3b-mix-448"
model = PaliGemmaForConditionalGeneration.from_pretrained(
    model_id, torch_dtype=torch.bfloat16, device_map="auto"
).eval()
processor = PaliGemmaProcessor.from_pretrained(model_id)

# Languages to iterate over
languages = ["en", "bn", "de", "ko", "ru", "zh"]

# Directory to save temp images (needed for this model)
os.makedirs("temp_images_vqa", exist_ok=True)

dataset = ds['xGQA_vqa']

for sample in tqdm(dataset, desc="Iterating samples"):
    try:
        image = sample["image"]
        sample_id = sample["sample_id"]

        # Save image locally if it isn't already saved
        image_path = f"temp_images_vqa/{sample_id}.jpg"
        if not os.path.exists(image_path):
            image.save(image_path)

        print(f"\n========== Sample ID: {sample_id} ==========")

        for lang in languages:
            question_col = f"question_{lang}"
            answer_col = f"answer_{lang}"

            # Check if the sample contains required fields
            if question_col not in sample or answer_col not in sample:
                print(f"[{lang}] Missing data.")
                continue

            question = sample[question_col]
            gt_answer = sample[answer_col]

            # Construct the prompt as per PaliGemma's VQA format
            formatted_prompt = f"answer {lang} {question}"

            # Load image into format expected by the processor
            loaded_image = load_image(image_path)

            # Prepare model inputs
            model_inputs = processor(
                text=formatted_prompt,
                images=loaded_image,
                return_tensors="pt"
            ).to(torch.bfloat16).to(model.device)

            input_len = model_inputs["input_ids"].shape[-1]

            # Perform inference
            with torch.inference_mode():
                generation = model.generate(
                    **model_inputs,
                    max_new_tokens=100,
                    do_sample=False
                )
                generation = generation[0][input_len:]

            # Decode generated tokens into string
            pred_answer = processor.decode(generation, skip_special_tokens=True)

            # Display results
            print(f"\n[{lang.upper()}]")
            print(f"Question: {question}")
            print(f"GT: {gt_answer}")
            print(f"Pred: {textwrap.fill(pred_answer, width=80)}")

        print("=" * 100)

    except Exception as e:
        print(f"Error processing sample {sample['sample_id']}: {e}")

    break

config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/75.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.07G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...


[KO]
Question: 날이 흐린가요?
GT: 아니, 분명해요.
Pred: no


You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.



[RU]
Question: Это пасмурная погода?
GT: Нет, это ясно.
Pred: no


Iterating samples:   0%|          | 0/9666 [00:19<?, ?it/s]


[ZH]
Question: 是阴天吗？
GT: 不，很清楚。
Pred: no
